In [1]:
#BERT MODEL CLASSIFICATION
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import tqdm
from tqdm import tqdm, trange
#WORK WITH TITLES

Using TensorFlow backend.
/Users/aashaiavadhani/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aashaiavadhani/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aashaiavadhani/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/aashaiavadhani/.local/lib/pytho

In [2]:
data_frameRealOnion = pd.read_csv('real_onion_good.csv')
data_frameRealOnion = data_frameRealOnion[:5000]

In [3]:
data_fakeOnion = pd.read_csv('better_fake_onion.csv')
data_fakeOnion = data_fakeOnion[:5000]

In [4]:
data_fakeOnion.dropna(inplace = True)
data_frameRealOnion.dropna(inplace = True)


In [6]:
#HAVE TO CREATE A DATAFRAME THAT HAS A COMBINATION OF BOTH DATA
frames = [data_frameRealOnion,data_fakeOnion]

#data_fakeOnion.head()
bigDataOnion = pd.concat(frames)
bigDataOnion = bigDataOnion.sample(frac=1).reset_index(drop=True)


In [23]:
#70%: Train, 30%: Test
data_length = len(bigDataOnion)
train_data_text = bigDataOnion[:80]
test_data_text = bigDataOnion[41:81]


In [24]:
#TOKENIZING
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [ ]:
keys = ['programmer','homemaker']


"This code gathers and returns the word embeddings for all the most similar words for the key phrases"
def getClusters(keys):
    embedCluster = []
    wordCluster = []
    for word in keys:
        embeddings,words = [], []
        modelSimilarWords = model.most_similar(word, topn=40)
        
#Need to iterate through the word and its embedding, thus temp var is needed
        for similar_word, temp in modelSimilarWords:
            words.append(similar_word)
            embeddings.append(model[similar_word])
            
        embedCluster.append(embeddings)
        wordCluster.append(words)
        
    return (embedCluster,wordCluster)


embedding = getClusters(keys)[0]
wordClustering = getClusters(keys)[1]

In [ ]:
#Setting up the clustering for the T-SNE

embedding_clusters = np.array(embedding)
n, m, k = embedding_clusters.shape

#Creating the TSNE model
#n iter allows for optimization of the model
#the random state produces the random number if the case for some values is None
tsne_model_en_2d = TSNE(perplexity=30, n_components=2, init = 'pca' ,n_iter=4000, random_state=35)

#transformes the embedding cluster model to 2 dimensions
transform2DEmbedding = tsne_model_en_2d.fit_transform(embedding_clusters.reshape(n * m, k))

#creating an array in order to 
array2DEmbedding = np.array(transform2DEmbedding)
finalEmbedding2D = array2DEmbedding.reshape(n, m, 2)



def plotTSNE(title, labels, embedding_clusters, word_clusters, a, filename=None):
    
    #creating the plot
    plt.figure(figsize=(17, 10))
    num_labels = len(labels)
    color = cm.rainbow(np.linspace(0, 1, num_labels))
    zippedData = zip(labels, embedding_clusters, word_clusters, color)
    for label, embeddings, words, color in zippedData:
        x, y = embeddings[:, 0], embeddings[:, 1]
        plt.scatter(x, y, c=color, alpha=a, label=label)
        
        #getting the word neighbors
        wordEnumeration = enumerate(words)            
        #labeling the embedding coordinates with their corresponding word
        for locationEmbedding, word in wordEnumeration:
            plt.annotate(word, alpha=0.5, xy=(x[locationEmbedding], y[locationEmbedding]), xytext=(5, 2),
                         textcoords='offset points', ha='right', va='bottom', size=10)

    plt.title(title), plt.legend(loc = 4)
    plt.show()

In [25]:
#print(len(train_data_text))
#print(train_data_text['label'])
#print(list(train_data_text['label']))
#print(list(train_data_text['title']))

train_data = []
for index_label in range(len(list(train_data_text['label']))):
    label = list(train_data_text['label'])[index_label]
    text = list(train_data_text['title'])[index_label]
    dictionary = {'title': text, 'label': label}
    train_data.append(dictionary)



test_data = []
for index_label in range(len(list(test_data_text['label']))):
    label = list(test_data_text['label'])[index_label]
    text = list(test_data_text['title'])[index_label]
    dictionary = {'title': text, 'label': label}
    test_data.append(dictionary)


#train_data = [{'title': text, 'label': type_data } for text in list(train_data_text['title']) for type_data in list(train_data_text['label'])]
#test_data = [{'title': text, 'label': type_data } for text in list(test_data_text['title']) for type_data in list(test_data_text['label'])]

In [26]:
train_texts, train_labels = list(zip(*map(lambda bigDataOnion: (bigDataOnion['title'], bigDataOnion['label']), train_data)))
test_texts, test_labels = list(zip(*map(lambda bigDataOnion: (bigDataOnion['title'], bigDataOnion['label']), test_data)))

In [27]:
print(len(train_texts))
print(len(test_texts))

80
40


In [28]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
print("hello")
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

hello


In [29]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

In [30]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [39]:
train_y = np.array(train_labels) == 0
test_y = np.array(test_labels) == 0
print(train_y)

[ True False False False False False  True False  True False False  True
 False False  True False  True False  True False False  True  True False
  True False False False  True False False  True False  True False False
 False  True False False False False False False False False  True False
 False  True  True False  True False  True  True False False False  True
 False False False  True False False  True False  True  True  True False
 False False  True  True False  True  True False]


In [40]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [41]:
#vectorizing each of the text data through the tensor masks
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [42]:
#converting the trained tokens into torch tensors 

train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [43]:
#setting up the BERT Classifier into training dataset, sampler, and the loader
BATCH_SIZE = 1
EPOCHS = 1
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [37]:
#The Actual Classifier within BERT as well as the training loop that uses a BCE loss as well as an Adam optimizer. It Implements the backward propation as well as a BCE loss to determine the error of the neural network error

bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in tqdm(range(EPOCHS)):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:  1
0/80.0 loss: 0.6663848161697388 
Epoch:  1
1/80.0 loss: 0.660313069820404 
Epoch:  1
2/80.0 loss: 0.7485995292663574 
Epoch:  1
3/80.0 loss: 0.7560811638832092 
Epoch:  1
4/80.0 loss: 0.7052189946174622 
Epoch:  1
5/80.0 loss: 0.6896471480528513 
Epoch:  1
6/80.0 loss: 0.7013253484453473 
Epoch:  1
7/80.0 loss: 0.724766306579113 
Epoch:  1
8/80.0 loss: 0.717884447839525 
Epoch:  1
9/80.0 loss: 0.729378092288971 
Epoch:  1
10/80.0 loss: 0.7271339784968983 
Epoch:  1
11/80.0 loss: 0.7209624797105789 
Epoch:  1
12/80.0 loss: 0.719561925301185 
Epoch:  1
13/80.0 loss: 0.7281773643834251 
Epoch:  1
14/80.0 loss: 0.7321641405423482 
Epoch:  1
15/80.0 loss: 0.7302170656621456 
Epoch:  1
16/80.0 loss: 0.7221196609384873 
Epoch:  1
17/80.0 loss: 0.7171893053584628 
Epoch:  1
18/80.0 loss: 0.7130195686691686 
Epoch:  1
19/80.0 loss: 0.7050286084413528 
Epoch:  1
20/80.0 loss: 0.6972067412875947 
Epoch:  1
21/80.0 loss: 0.6934465305371718 
Epoch:  1
22/80.0 loss: 0.6927718986635623 
Epo

100%|██████████| 1/1 [04:13<00:00, 253.90s/it]

Epoch:  1
79/80.0 loss: 0.666093573719263 


In [38]:
#Using the BERT classifier on the test data

bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       1.00      0.12      0.21        17
        True       0.61      1.00      0.75        23

    accuracy                           0.62        40
   macro avg       0.80      0.56      0.48        40
weighted avg       0.77      0.62      0.52        40



This was only conducted on 40 articles with a training 